In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, accuracy_score, auc, recall_score, confusion_matrix


In [2]:
xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     period = "max",
                     interval = "1d").reset_index()

[*********************100%%**********************]  1 of 1 completed


In [3]:
df_xmr

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600
1,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000
2,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000
3,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000
4,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000
...,...,...,...,...,...,...,...
2164,2023-10-13,152.339523,154.368774,151.051971,153.590515,153.590515,51282283
2165,2023-10-14,153.583801,154.282990,151.850967,152.833160,152.833160,33344438
2166,2023-10-15,152.833466,153.465881,152.173126,152.448990,152.448990,31895675
2167,2023-10-16,152.437531,153.522949,151.155411,152.544098,152.544098,64382308


In [4]:
euro_usd = yf.download(tickers = "EURUSD=X",
                     start = '2017-11-09',
                     end = '2023-10-17',
                     interval = "1d").reset_index()
euro_usd = euro_usd.rename(columns={'Open': 'euro_usd_open',
                                    'High': 'euro_usd_high',
                                    'Low': 'euro_usd_low',
                                    'Close': 'euro_usd_close',
                                    'Adj Close': 'euro_usd_adj_close',
                                    'Volume': 'euro_usd_volume'})

[*********************100%%**********************]  1 of 1 completed


In [5]:
euro_usd

,Date,euro_usd_open,euro_usd_high,euro_usd_low,euro_usd_close,euro_usd_adj_close,euro_usd_volume
0,2017-11-09,1.159555,1.165257,1.158641,1.159689,1.159689,0
1,2017-11-10,1.164524,1.167515,1.162399,1.164687,1.164687,0
2,2017-11-13,1.166113,1.167515,1.163873,1.166113,1.166113,0
3,2017-11-14,1.166494,1.177413,1.166330,1.166494,1.166494,0
4,2017-11-15,1.179176,1.185944,1.178606,1.179287,1.179287,0
...,...,...,...,...,...,...,...
1541,2023-10-10,1.057888,1.061954,1.055598,1.057888,1.057888,0
1542,2023-10-11,1.060479,1.063355,1.059277,1.060479,1.060479,0
1543,2023-10-12,1.062428,1.063909,1.054596,1.062428,1.062428,0
1544,2023-10-13,1.053674,1.055855,1.049615,1.053674,1.053674,0


In [6]:
df = df_xmr.copy()

In [7]:
df = df.merge(euro_usd, on="Date", how="outer")

In [8]:
print(df.isna().sum(), '\n')
print('Shape of DataFrame:')
print(df.shape)

Date                    0
Open                    0
High                    0
Low                     0
Close                   0
Adj Close               0
Volume                  0
euro_usd_open         623
euro_usd_high         623
euro_usd_low          623
euro_usd_close        623
euro_usd_adj_close    623
euro_usd_volume       623
dtype: int64 

Shape of DataFrame:
(2169, 13)


In [9]:
rows_with_nan = df[df.isna().any(axis=1)]
nun_days = rows_with_nan['Date'].dt.day_name().value_counts()
print(nun_days)

Date
Saturday     310
Sunday       310
Thursday       1
Wednesday      1
Tuesday        1
Name: count, dtype: int64


In [10]:
to_fill_cols = list(df.columns)
for col in range(7, len(to_fill_cols)):
    df[to_fill_cols[col]] = df[to_fill_cols[col]].interpolate(method='polynomial', order=2)

In [11]:
print(df.isna().sum(), '\n')
print('Shape of DataFrame:')
print(df.shape)

Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Adj Close             0
Volume                0
euro_usd_open         1
euro_usd_high         1
euro_usd_low          1
euro_usd_close        1
euro_usd_adj_close    1
euro_usd_volume       1
dtype: int64 

Shape of DataFrame:
(2169, 13)


In [12]:
df = df.dropna()

In [13]:
print(df.shape)

(2168, 13)


In [14]:
# labeling rows based on close price of day before

df.loc[0, "price_increase"] = 0
for i in range(1, len(df)):
    df.loc[i, "price_increase"] = 1 if df.loc[i-1, "Close"] < df.loc[i, "Close"] else 0

In [15]:
# split data in two part one for testing and one for training and validating

date_range = pd.date_range(start="2022-09-08", end="2023-09-07", freq='D')
test_df = df[df["Date"].isin(date_range)]
train_df = df.drop(index=test_df.index)

test_df = test_df.drop('Date', axis=1)
train_df = train_df.drop('Date', axis=1)

X = train_df.drop('price_increase', axis=1)
y = train_df['price_increase']

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

X_test = test_df.drop('price_increase', axis=1)
y_test = test_df['price_increase']

logistic regression with xmr and euro-usd data

In [16]:
param_grid = [
    {'solver': ['liblinear'], 'penalty': ['l1', 'l2'], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['lbfgs'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['newton-cg', 'newton-cholesky'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['sag'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['saga'], 'penalty': ['l1', 'l2', 'elasticnet', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]}
]

clf = Pipeline([('scaler', StandardScaler()),
                ('lr', GridSearchCV(LogisticRegression(),
                              param_grid=param_grid,
                              cv=5,
                              refit=True,
                              verbose=10))])
clf.fit(X_train, y_train)

Fitting 5 folds for each of 448 candidates, totalling 2240 fits
[CV 1/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 1/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.984 total time=   0.0s
[CV 2/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 2/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.945 total time=   0.0s
[CV 3/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 3/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.921 total time=   0.0s
[CV 4/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 4/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.940 total time=   0.0s
[CV 5/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 5/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.929 total time=   0.0s
[CV 1/5; 2/448] START C=0.1, pe

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.984 total time=   0.0s
[CV 2/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 2/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.992 total time=   0.1s
[CV 3/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 3/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.976 total time=   0.0s
[CV 4/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 4/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.984 total time=   0.1s
[CV 5/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 5/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.992 total time=   0.0s
[CV 1/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 1/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.984 total time=   0

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.992 total time=   0.1s
[CV 3/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 3/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.976 total time=   0.1s
[CV 4/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 4/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.984 total time=   0.1s
[CV 5/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 5/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.992 total time=   0.1s
[CV 1/5; 12/448] START C=1, penalty=l1, solver=liblinear, tol=1e-06.............


...

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which mean

[CV 2/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.858 total time=   0.0s
[CV 3/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 3/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.960 total time=   0.0s
[CV 4/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 4/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.821 total time=   0.0s
[CV 5/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 5/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.813 total time=   0.0s
[CV 1/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 1/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.889 total time=   0.0s
[CV 2/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 2/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-0

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which mean

[CV 3/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.960 total time=   0.0s
[CV 4/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 4/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.821 total time=   0.0s
[CV 5/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 5/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.813 total time=   0.0s


Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 GridSearchCV(cv=5, estimator=LogisticRegression(),
                              param_grid=[{'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2'],
                                           'solver': ['liblinear'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['lbfgs'],
                                           'to...
                                           'solver': ['newton-cg',
                                                      'newton-cholesky'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['sag'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2', 'elasticnet',
                                                       None],
                                           'solver': ['saga'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]}],
                              verbose=10))])

In [17]:
clf['lr'].best_estimator_
print(clf['lr'].best_score_)

0.985733107472238


Validation 

In [18]:
y_valid_pred = clf.predict(X_valid)
y_test_pred = clf.predict(X_test)

scores_1 ={"f1 Score" : f1_score(y_valid, y_valid_pred),
"accuracy score" : accuracy_score(y_valid, y_valid_pred),
"recall score" : recall_score(y_valid, y_valid_pred),
"precision score" : precision_score(y_valid, y_valid_pred)}

conf_matrix_1 = confusion_matrix(y_valid, y_valid_pred)

Evaluation with test dataset

In [19]:
y_test_pred = clf.predict(X_test)

test_score_1 = {"f1 Score" : f1_score(y_test, y_test_pred),
                "accuracy score" : accuracy_score(y_test, y_test_pred),
                "recall score" : recall_score(y_test, y_test_pred),
                "precision score" : precision_score(y_test, y_test_pred)}

test_conf_matrix_1 = confusion_matrix(y_test, y_test_pred)

logistic regression with xmr data

In [20]:
df = df_xmr.copy()

In [21]:
df.loc[0, "price_increase"] = 0
for i in range(1, len(df)):
    df.loc[i, "price_increase"] = 1 if df.loc[i-1, "Close"] < df.loc[i, "Close"] else 0

In [22]:
print(df.shape)

(2169, 8)


In [23]:
date_range = pd.date_range(start="2022-09-08", end="2023-09-07", freq='D')
test_df = df[df["Date"].isin(date_range)]
train_df = df.drop(index=test_df.index)

test_df = test_df.drop('Date', axis=1)
train_df = train_df.drop('Date', axis=1)

X = train_df.drop('price_increase', axis=1)
y = train_df['price_increase']

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

X_test = test_df.drop('price_increase', axis=1)
y_test = test_df['price_increase']

In [24]:
param_grid = [
    {'solver': ['liblinear'], 'penalty': ['l1', 'l2'], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['lbfgs'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['newton-cg', 'newton-cholesky'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['sag'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['saga'], 'penalty': ['l1', 'l2', 'elasticnet', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]}
]

clf = Pipeline([('scaler', StandardScaler()),
                ('lr', GridSearchCV(LogisticRegression(),
                              param_grid=param_grid,
                              cv=5,
                              refit=True,
                              verbose=10))])
clf.fit(X_train, y_train)

Fitting 5 folds for each of 448 candidates, totalling 2240 fits
[CV 1/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 1/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.976 total time=   0.0s
[CV 2/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 2/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.953 total time=   0.0s
[CV 3/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 3/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.925 total time=   0.0s
[CV 4/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 4/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.968 total time=   0.0s
[CV 5/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 5/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.964 total time=   0.0s
[CV 1/5; 2/448] START C=0.1, pe

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.972 total time=   0.0s
[CV 4/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 4/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.992 total time=   0.1s
[CV 5/5; 10/448] START C=1, penalty=l1, solver=liblinear, tol=0.0001............
[CV 5/5; 10/448] END C=1, penalty=l1, solver=liblinear, tol=0.0001;, score=0.996 total time=   0.1s
[CV 1/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 1/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.996 total time=   0.1s
[CV 2/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............


c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.992 total time=   0.1s
[CV 3/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 3/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.972 total time=   0.2s
[CV 4/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 4/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.992 total time=   0.1s
[CV 5/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 5/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.996 total time=   0.2s
[CV 1/5; 12/448] START C=1, penalty=l1, solver=liblinear, tol=1e-06.............
[CV 1/5; 12/448] END C=1, penalty=l1, solver=liblinear, tol=1e-06;, score=0.996 total time=   0.1s
[CV 2/5; 12/448] START C=1, penalty=l1, solver=liblinear, tol=1e-06.............
[CV 2/5; 12/448] END C=1, penalty=l1, solver=liblinear, tol=1e-06;, score=0.992 total time=   0.2s
[

...

[CV 1/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.913 total time=   0.0s
[CV 2/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 2/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.885 total time=   0.0s
[CV 3/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 3/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.925 total time=   0.0s
[CV 4/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 4/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.917 total time=   0.0s
[CV 5/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 5/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.917 total time=   0.0s
[CV 1/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 1/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-0

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which mean

[CV 2/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.885 total time=   0.0s
[CV 3/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 3/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.925 total time=   0.0s
[CV 4/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 4/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.913 total time=   0.0s
[CV 5/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 5/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.917 total time=   0.0s


c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [0.95719932 0.95719932 0.95719932 0.95719932 0.78449087 0.78449087
 0.78449087 0.78449087 0.98969195 0.98969195 0.98969195 0.98969195
 0.94690382 0.94690382 0.94690382 0.94690382 0.9888983  0.9888983
 0.9888983  0.9888983  0.97464082 0.97464082 0.97464082 0.97464082
 0.9920729  0.99286655 0.99127925 0.99127925 0.98493318 0.98493318
 0.98493318 0.98493318 0.99127925 0.99127925 0.99127925 0.98969195
 0.9904856  0.9904856  0.9904856  0.9904856  0.9888983  0.9904856
 0.9904856  0.99127925 0.99048874 0.99048874 0.99048874 0.99048874
 0.9904856  0.99128239 0.9920729  0.9904856  0.99048874 0.99048874
 0.99048874 0.99048874 0.9920729  0.99127925 0.9904856  0.99048874
 0.99048874 0.99128239 0.99128239 0.99128239 0.77815108 0.77815108
 0.77815108 0.77815108 0.99128239 0.99128239 0.99128239 0.99128239
 0.94611017 0.94611017 

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 GridSearchCV(cv=5, estimator=LogisticRegression(),
                              param_grid=[{'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2'],
                                           'solver': ['liblinear'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['lbfgs'],
                                           'to...
                                           'solver': ['newton-cg',
                                                      'newton-cholesky'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['sag'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2', 'elasticnet',
                                                       None],
                                           'solver': ['saga'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]}],
                              verbose=10))])

In [25]:
clf['lr'].best_estimator_
print(clf['lr'].best_score_)

0.9928665537361191


In [26]:
y_valid_pred = clf.predict(X_valid)
y_test_pred = clf.predict(X_test)

scores_2 ={"f1 Score" : f1_score(y_valid, y_valid_pred),
"accuracy score" : accuracy_score(y_valid, y_valid_pred),
"recall score" : recall_score(y_valid, y_valid_pred),
"precision score" : precision_score(y_valid, y_valid_pred)}

conf_matrix_2 = confusion_matrix(y_valid, y_valid_pred)

Evaluating with test data

In [27]:
y_test_pred = clf.predict(X_test)

test_score_2 = {"f1 Score" : f1_score(y_test, y_test_pred),
                "accuracy score" : accuracy_score(y_test, y_test_pred),
                "recall score" : recall_score(y_test, y_test_pred),
                "precision score" : precision_score(y_test, y_test_pred)}

test_conf_matrix_2 = confusion_matrix(y_test, y_test_pred)

regression with xmr, auro_usd, and sp 500 datasets

In [28]:
sp500 = yf.download(tickers = "^GSPC",
                    start = '2017-11-09',
                    end = '2023-10-15',
                    interval = "1d").reset_index()
sp500 = sp500.rename(columns={'Open': 'sp500_open', 'High': 'sp500_high', 'Low': 'sp500_low', 'Close': 'sp500_close', 'Adj Close': 'sp500_adj_close', 'Volume': 'sp500_volume'})

[*********************100%%**********************]  1 of 1 completed


In [29]:
df = df.merge(sp500, on="Date", how="outer")

In [30]:
df.shape

(2169, 14)

In [31]:
print(df.isna().sum(), '\n')
print('Shape of DataFrame:')
print(df.shape)


Date                 0
Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
price_increase       0
sp500_open         678
sp500_high         678
sp500_low          678
sp500_close        678
sp500_adj_close    678
sp500_volume       678
dtype: int64 

Shape of DataFrame:
(2169, 14)


In [32]:
rows_with_nan = df[df.isna().any(axis=1)]
nun_days = rows_with_nan['Date'].dt.day_name().value_counts()
print(nun_days)

Date
Saturday     310
Sunday       310
Monday        33
Friday        10
Thursday       7
Wednesday      4
Tuesday        4
Name: count, dtype: int64


In [33]:
to_fill_cols = list(df.columns)
for col in range(14, len(to_fill_cols)):
    df[to_fill_cols[col]] = df[to_fill_cols[col]].interpolate(method='polynomial', order=2)

In [34]:
print(df.isna().sum())

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
price_increase       0
sp500_open         678
sp500_high         678
sp500_low          678
sp500_close        678
sp500_adj_close    678
sp500_volume       678
dtype: int64


In [35]:
df = df.dropna()

In [36]:
date_range = pd.date_range(start="2022-09-08", end="2023-09-07", freq='D')
test_df = df[df["Date"].isin(date_range)]
train_df = df.drop(index=test_df.index)

test_df = test_df.drop('Date', axis=1)
train_df = train_df.drop('Date', axis=1)

X = train_df.drop('price_increase', axis=1)
y = train_df['price_increase']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

X_test = test_df.drop('price_increase', axis=1)
y_test = test_df['price_increase']

In [37]:
param_grid = [
    {'solver': ['liblinear'], 'penalty': ['l1', 'l2'], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['lbfgs'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['newton-cg', 'newton-cholesky'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['sag'], 'penalty': ['l2', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]},
    {'solver': ['saga'], 'penalty': ['l1', 'l2', 'elasticnet', None], 'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]}
]

clf = Pipeline([('scaler', StandardScaler()),
                ('lr', GridSearchCV(LogisticRegression(),
                              param_grid=param_grid,
                              cv=5,
                              refit=True,
                              verbose=10))])
clf.fit(X_train, y_train)

Fitting 5 folds for each of 448 candidates, totalling 2240 fits
[CV 1/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 1/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.839 total time=   0.0s
[CV 2/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 2/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.833 total time=   0.0s
[CV 3/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 3/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.920 total time=   0.0s
[CV 4/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 4/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.971 total time=   0.0s
[CV 5/5; 1/448] START C=0.1, penalty=l1, solver=liblinear, tol=0.001............
[CV 5/5; 1/448] END C=0.1, penalty=l1, solver=liblinear, tol=0.001;, score=0.983 total time=   0.0s
[CV 1/5; 2/448] START C=0.1, pe

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.988 total time=   0.1s
[CV 5/5; 11/448] START C=1, penalty=l1, solver=liblinear, tol=1e-05.............
[CV 5/5; 11/448] END C=1, penalty=l1, solver=liblinear, tol=1e-05;, score=0.977 total time=   0.0s
[CV 1/5; 12/448] START C=1, penalty=l1, solver=liblinear, tol=1e-06.............


c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5; 12/448] END C=1, penalty=l1, solver=liblinear, tol=1e-06;, score=0.966 total time=   0.1s
[CV 2/5; 12/448] START C=1, penalty=l1, solver=liblinear, tol=1e-06.............


...

c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which mean

[CV 5/5; 446/448] END C=1000000.0, penalty=None, solver=saga, tol=0.0001;, score=0.913 total time=   0.0s
[CV 1/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 1/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.828 total time=   0.0s
[CV 2/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 2/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.920 total time=   0.0s
[CV 3/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 3/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.833 total time=   0.0s
[CV 4/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......
[CV 4/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.867 total time=   0.0s
[CV 5/5; 447/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-05.......


c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\mehrdad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which mean

[CV 5/5; 447/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-05;, score=0.913 total time=   0.0s
[CV 1/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 1/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.828 total time=   0.0s
[CV 2/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 2/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.920 total time=   0.0s
[CV 3/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 3/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.833 total time=   0.0s
[CV 4/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 4/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-06;, score=0.867 total time=   0.0s
[CV 5/5; 448/448] START C=1000000.0, penalty=None, solver=saga, tol=1e-06.......
[CV 5/5; 448/448] END C=1000000.0, penalty=None, solver=saga, tol=1e-0

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr',
                 GridSearchCV(cv=5, estimator=LogisticRegression(),
                              param_grid=[{'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2'],
                                           'solver': ['liblinear'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['lbfgs'],
                                           'to...
                                           'solver': ['newton-cg',
                                                      'newton-cholesky'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l2', None],
                                           'solver': ['sag'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]},
                                          {'C': [0.1, 1, 10.0, 100.0, 1000.0,
                                                 10000.0, 100000.0, 1000000.0],
                                           'penalty': ['l1', 'l2', 'elasticnet',
                                                       None],
                                           'solver': ['saga'],
                                           'tol': [0.001, 0.0001, 1e-05,
                                                   1e-06]}],
                              verbose=10))])

In [38]:
y_valid_pred = clf.predict(X_valid)
y_test_pred = clf.predict(X_test)

scores_3 ={"f1 Score" : f1_score(y_valid, y_valid_pred),
"accuracy score" : accuracy_score(y_valid, y_valid_pred),
"recall score" : recall_score(y_valid, y_valid_pred),
"precision score" : precision_score(y_valid, y_valid_pred)}

conf_matrix_3 = confusion_matrix(y_valid, y_valid_pred)

evaluating

In [39]:
y_test_pred = clf.predict(X_test)

test_score_3 = {"f1 Score" : f1_score(y_test, y_test_pred),
                "accuracy score" : accuracy_score(y_test, y_test_pred),
                "recall score" : recall_score(y_test, y_test_pred),
                "precision score" : precision_score(y_test, y_test_pred)}

test_conf_matrix_3 = confusion_matrix(y_test, y_test_pred)

In [43]:
scores_1 # on validation data of xmr and euro-usd

{'f1 Score': 0.9911504424778762,
 'accuracy score': 0.9907578558225508,
 'recall score': 0.99644128113879,
 'precision score': 0.9859154929577465}

In [46]:
scores_2 # on validation data of xmr

{'f1 Score': 0.9891696750902527,
 'accuracy score': 0.988929889298893,
 'recall score': 0.9856115107913669,
 'precision score': 0.9927536231884058}

In [47]:
scores_3 # on validation date of xmr, euro-usd, sp 500

{'f1 Score': 0.9787234042553191,
 'accuracy score': 0.978494623655914,
 'recall score': 0.9787234042553191,
 'precision score': 0.9787234042553191}

In [40]:
test_score_1 # on test data of xmr and euro-usd

{'f1 Score': 0.9804878048780488,
 'accuracy score': 0.9780821917808219,
 'recall score': 1.0,
 'precision score': 0.9617224880382775}

In [41]:
test_score_2 # on test data data of xmr

{'f1 Score': 0.9877149877149877,
 'accuracy score': 0.9863013698630136,
 'recall score': 1.0,
 'precision score': 0.9757281553398058}

In [42]:
test_score_3 # on test data  of xmr, euro-usd, sp 500

{'f1 Score': 0.9660377358490566,
 'accuracy score': 0.9641434262948207,
 'recall score': 1.0,
 'precision score': 0.9343065693430657}